In [26]:
from googletrans import Translator
import os

SRC = "en"
DEST = "tr"
START_IDX = 0
TRANSLATE_LIMIT = -1 # unlimited
DATA_PATH = "D:/C12M/cc12m.tsv"
OUTPUT_PATH = "D:/C12M/cc12m_tr.tsv"
RESPECT_DURATION = 1
TRANSLATE_BATCH_SIZE = 10
translator = Translator()

def batch_translate(batch, translator):
	return translator.translate(batch, src=SRC, dest=DEST)

def to_tsv(url, en, tr):
	return f"{url}\t{en}\t{tr}\n"

In [27]:
if os.path.exists(OUTPUT_PATH):
	with open(OUTPUT_PATH, "r", encoding="utf-8") as dest:
		for line in dest:
			START_IDX += 1
	print(f"Found {START_IDX} entries.")

with open(DATA_PATH, "r", encoding="utf-8") as src, open(OUTPUT_PATH, "a", encoding="utf-8") as dest:
	batch = [""] * TRANSLATE_BATCH_SIZE
	active_lines = [""] * TRANSLATE_BATCH_SIZE
	index = 0
	translate_counter = 0
	for i, line in enumerate(src):
		if i < START_IDX:
			continue
		url, text = line.strip().split("\t")
		index = translate_counter % TRANSLATE_BATCH_SIZE
		active_lines[index] = (url, text)
		batch[index] = text
		translate_counter += 1
		if translate_counter == TRANSLATE_LIMIT:
			break
		if index == (TRANSLATE_BATCH_SIZE-1):
			translations = batch_translate(batch, translator)
			for i, translation in enumerate(translations):
				dest.write(to_tsv(active_lines[i][0], active_lines[i][1], translation.text))
			index = -1	# index is different than -1 after the loop, we need to translate remainder lines as well
	if index != -1:
		translations = batch_translate(batch[:index+1], translator)
		for i, translation in enumerate(translations):
			dest.write(to_tsv(active_lines[i][0], active_lines[i][1], translation.text))

Found 9240 entries.
